## 1. Importación de las bibliotecas necesarias

In [ ]:
# Instalar geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [1]:
import ee
import geemap as emap
Map = emap.Map()
Map.add_basemap('ROADMAP')

## 2. Visualización y descarga de índices NDVI MODIS para la provincia de Lima

### 2.1 Visualización de los máximos NDVI

In [2]:
# Establecemos la region de mapeo (provincia de Lima)
region = ee.FeatureCollection('users/geoyons/tugee/LimaProv')
mapLomas = ee.FeatureCollection('users/geoyons/tugee/Lomas_Modis')

# Definimos la coleccion
col = 'MODIS/006/MOD13Q1'

# Definimos las bandas
bands = ['NDVI']

# Funcion de filtro de pixeles contaminados usando la banda de fiabilidad
def maskSQA(image):
    mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
    imageMask = image.updateMask(mask)
    return imageMask
    
# Filtrado de la coleccion MODIS
colMODIS = ee.ImageCollection(col)\
        .filterDate('2018-01-01', '2018-12-31')\
        .filterBounds(region)\
        .map(maskSQA)\
        .reduce(ee.Reducer.max()).divide(10000).clip(mapLomas)\
        .reproject(crs = 'EPSG:4326', scale = 250)

# Seleccion del indice NDVI
ndvi = colMODIS.select([0], bands)

# Definimos los parametros de visualizacion
visParams = {
  'min': 0.02,
  'max': 0.4,
  'palette': ['FFFFFF','CC9966','CC9900','996600', '33CC00', '009900','006600']
}

# Visualizamos el ndvi y la region de estudio
Map.centerObject(region, 9)
Map.addLayer(region, {}, 'Provincia de Lima', True)
Map.addLayer(ndvi, visParams, 'NDVImax MODIS 2018')
Map.add_colorbar(visParams, label="Normalized Difference Vegetation Index (NDVI)")
Map

Map(center=[-12.020761229806144, -76.91462520735031], controls=(WidgetControl(options=['position'], widget=HBo…

### 2.2 Descarga de los máximos NDVI

In [ ]:
# Seccion de codigo para descarga automatica de indices MODIS
col = 'MODIS/006/MOD13Q1'

bands = ['NDVI']

# Establecemos la region de mapeo (provincia de Lima)
region = ee.FeatureCollection('users/geoyons/tugee/LimaProv')

def getNDVImodis(ini_date, end_date, roi, folder):
    '''
    Esta funcion permite descargar automáticamente imagenes MODIS
    directamente a una carpeta de Google Drive personal.
    
    Parametros:
    
        ini_date: Fecha de inicio
        
        ini_end: Fecha de fin
        
        roi: region de interes (FeatureCollection)
        
        folder: Nombre de la carpeta donde se descargara las imagenes
    
    Retura:
        Imagenes de indices descargados en .tif.
    '''
    
    def maskSQA(image):
        mask = image.select('SummaryQA').eq(0).Or(image.select('SummaryQA').eq(1))
        imageMask = image.updateMask(mask)
        return imageMask
    
    ndvi = ee.ImageCollection(col)\
        .filterDate(ini_date, end_date)\
        .filterBounds(roi)\
        .map(maskSQA)\
        .reduce(ee.Reducer.max())\
        .divide(10000)\
        .clip(roi)\
        .select([0], bands)
    
    # Descargar directamente a Google Drive
    task = ee.batch.Export.image(ndvi.select('NDVI'), 
                                 'NDVI_MODIS_' + ini_date + '_' + end_date + '_median', 
                                 {'region': region.geometry(),
                                  'scale': 250,
                                  'maxPixels': 1e13,
                                  'folder': folder,
                                  'fileFormat': 'GeoTIFF',
                                  'crs': 'EPSG:4326'})
    print('Descargando la imagen:'+'NDVI_MODIS_'+ini_date+'_'+end_date+'_max'+'.tif')
    task.start()

# Ejemplo de descargar para el año 2020
getNDVImodis(ini_date = '2020-01-01', end_date = '2020-12-31', roi = region, folder = 'MOD13Q1_NDVI')